<a href="https://colab.research.google.com/github/Ahmed8501/Masters/blob/main/notebooks/masters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Ahmed8501/Masters.git


Cloning into 'Masters'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 12.56 KiB | 3.14 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
cd Masters/notebooks


/content/Masters/notebooks
